###**Кодирование**


In [ ]:
from PIL import Image,ImageDraw

def extracting_pixel(file_name, channel='pixel'):
  # with автоматически закроет файл после работы
  with Image.open(file_name) as img:#.open для чтения изображения из файла
      obj=img.load()#для чтения изображения в память, чтобы файл можно было закрыть
      num_row,num_col=img.size #.size получаем размеры изображения (16 на 16 пиксилей)
      blue=[]#для хранение синего канала каждого пикселя
      # проходить изображение попиксельно
      for row in range(num_row):
          for col in range(num_col):
            if channel == 'pixel':
              blue.append(obj[row,col])
            else:
              blue.append(obj[row,col][channel])#[channel] указывает на канал (синий=2); [row,col] координаты пиксиля; obj изображение
  return blue

def group(listing, count):
  return [listing[i:i + count] for i in range(0, len(listing), count)]


def NZB (num):#наименьший значащий бит(НЗБ)
  return bin(num)[-1]

def find_index(value, lst):
  for tpl in lst:
    if value == tpl[1]:
      return tpl[0]

def find_position(value, lst):
  for i in range(len(lst)):
    if value == lst[i][1]:
      return i

def find_new_value(bit, value, lst):
  position_up=position_down=find_position(value, lst)
  value_up=value_down=0

  for i in lst:
    if position_up+1 < len(lst):
      position_up+=1
      if NZB(lst[position_up][0]) == bit:
        value_up=lst[position_up][1]-value
        break

  for i in lst:
    if position_down-1 > 0:
      position_down-=1
      if NZB(lst[position_down][0]) == bit:
        value_down=value-lst[position_down][1]
        break

  if value_down <= value_up:
    return lst[position_down][1]
  else:
    return lst[position_up][1]

def find_new_blue(bin_data, blue, blue_sort):
  new_blue=[]
  for index, bit in enumerate(bin_data):
    if NZB(find_index(blue[index], blue_sort))==bit:
      new_blue.append(blue[index])
    else:
      new_blue.append(find_new_value(bit, blue[index], blue_sort))
  return new_blue

def newImg(size, list_pixel):
    img = Image.new('RGB', size)
    num_row,num_col=size
    i=0
    for row in range(num_row):
        for col in range(num_col):
            img.putpixel((row,col), list_pixel[i])
            i+=1
    img.save('sqr.bmp')
    return img

In [ ]:
container = "/content/drive/MyDrive/Контейнер.bmp"
blue = extracting_pixel(container, 2)#извлечение синего канала
# Создание уникального кортежа типа (порядковый номер, значение синего канала);
blue_unique=[(index,value) for index,value in enumerate(list(set(blue)))]#list(set(blue)) оставляет только уникальные значения синего канала;
blue_sort=sorted(blue_unique, key=lambda i: i[1])#сортировка синего канала по возрастанию

In [ ]:
cvz = '/content/drive/MyDrive/QRcode.bmp'
with open(cvz, 'rb') as file:
  byte = file.read()#чтение байтов файла (b'BM6\x03\x00\x00\x00\x00\x00\x006\')

byte_list=group(byte.hex(), 2)#.hex() переводит из BM6\x03\x00\x00\  в 424d3603; групирует последовательность байт по 2 (['42', '4d', '36', '03']);

bin_text= ''.join([bin(int(byte, base=16))[2:].zfill(8) for byte in byte_list])#bin(int(byte, base=16)) переводит каждый байт в биты ('42'->'01000010'); [2:] обрезает 0b в тексте '0b101001'; .zfill(8) дополняет нулями до 8 цифр (101001 -> 00101001); ''.join() соеденяет значения в списке (['01000010', '01001101'] -> 0100001001001101)

In [ ]:
new_blue = find_new_blue(bin_text, blue, blue_sort)
pixel_image = extracting_pixel(container)

new_pixel_image=[]
length=len(new_blue)
head=pixel_image[:length]# Задаём конечным индексом, конец не включается
body=pixel_image[length:]# Задаём начальным индексом, начало включается

for index, pixel in enumerate(head):
  r,g,b=pixel
  new_pixel = (r, g, new_blue[index])
  new_pixel_image.append(new_pixel)

new_pixel_image+=body

with Image.open(container) as img:
  size=img.size
wallpaper = newImg(size, new_pixel_image)

---
###**Декодирование**

In [ ]:
blue_list = extracting_pixel('sqr.bmp', 2)

bin_str=''
for blue in blue_list:
  bit=NZB(find_index(blue, blue_sort))
  bin_str+=bit

group_bits=group(bin_str, 8)
byte_list = bytes([int(bits, 2) for bits in group_bits])

with open("test.bmp", "wb") as f:
    f.write(byte_list)